## Import thư viện PyTorch

In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

## Thiết lập device

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device 

device(type='cuda')

## Đặt các siêu tham số (Hyper-parameters)

In [3]:
# Kịch thước đầu vào
input_size = 784

# Kịch thước ẩn
hidden_size = 500

# Số lượng lớp
num_classes = 10

# Số lượng epochs
num_epochs = 5

# Kích thước mỗi batch
batch_size = 100

# Tốc độ học - Learning rate
learning_rate = 0.001

## Load bộ dữ liệu MNTST

In [4]:
# Dữ liệu tập huấn luyện
train_dataset = torchvision.datasets.MNIST(root='../../data', 
                                           train=True, 
                                           transform=transforms.ToTensor(),
                                           download=True)

# Dữ liệu tập kiểm tra
test_dataset = torchvision.datasets.MNIST(root='../../data', 
                                          train=False, 
                                          transform=transforms.ToTensor())

## DataLoader

In [5]:
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

## Mô hình Feedforward Neural Network

In [6]:
# Fully connected neural network with one hidden layer
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes):
        super(NeuralNet, self).__init__()
        self.fc1 = nn.Linear(input_size, hidden_size) 
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, num_classes)  
    
    def forward(self, x):
        out = self.fc1(x)
        out = self.relu(out)
        out = self.fc2(out)
        return out

In [7]:
model = NeuralNet(input_size, hidden_size, num_classes).to(device)

## Hàm mất mát (loss function) và bộ tối ưu hóa (optimizer)

In [8]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)  

## Huấn luyện mô hình

In [9]:
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):  
        # ịnh dáng lại (reshape) dữ liệu thành  và chuyển dữ liệu sang device
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        
        # Lan truyền tiến (Forward pass) và tính toán mất mát
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Lan truyền ngược (Backward pass) và tối ưu
        ## Đăt zero_grad để không tính toán đạo hàm
        optimizer.zero_grad()
        ## Gọi backward()
        loss.backward()
        ## Tối ưu
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.4609
Epoch [1/5], Step [200/600], Loss: 0.3561
Epoch [1/5], Step [300/600], Loss: 0.2517
Epoch [1/5], Step [400/600], Loss: 0.1541
Epoch [1/5], Step [500/600], Loss: 0.1791
Epoch [1/5], Step [600/600], Loss: 0.2028
Epoch [2/5], Step [100/600], Loss: 0.1130
Epoch [2/5], Step [200/600], Loss: 0.0874
Epoch [2/5], Step [300/600], Loss: 0.1538
Epoch [2/5], Step [400/600], Loss: 0.0552
Epoch [2/5], Step [500/600], Loss: 0.0518
Epoch [2/5], Step [600/600], Loss: 0.1469
Epoch [3/5], Step [100/600], Loss: 0.0647
Epoch [3/5], Step [200/600], Loss: 0.0560
Epoch [3/5], Step [300/600], Loss: 0.0513
Epoch [3/5], Step [400/600], Loss: 0.0705
Epoch [3/5], Step [500/600], Loss: 0.0372
Epoch [3/5], Step [600/600], Loss: 0.0277
Epoch [4/5], Step [100/600], Loss: 0.0149
Epoch [4/5], Step [200/600], Loss: 0.0753
Epoch [4/5], Step [300/600], Loss: 0.0628
Epoch [4/5], Step [400/600], Loss: 0.1565
Epoch [4/5], Step [500/600], Loss: 0.0294
Epoch [4/5], Step [600/600], Loss:

## Kiểm tra mô hình

In [10]:
# Ở giai đoạn kiểm tra mô hình, không cần tính đạo hàm nên sử dụng with torch.no_grad()
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.reshape(-1, 28*28).to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: {} %'.format(100 * correct / total))

Accuracy of the network on the 10000 test images: 97.74 %


## Lưu mô hình (model checkpoint)

In [11]:
torch.save(model.state_dict(), 'model.ckpt')